# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])  # Download necessary NLTK resources

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db') 
df = pd.read_sql_table('disaster_messages', engine) 
X = df.message.values 
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """Tokenize and clean text data."""
    # Regular expression for detecting URLs in text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)  # Detect URLs in text
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder") 

    tokens = word_tokenize(text)  # Tokenize text into words
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]  # Lemmatize and clean tokens
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def build_pipeline():
    
    return Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = build_pipeline()
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = model.predict(X_test)

# Convert Y_test and Y_pred to DataFrames for easier column access
Y_test = pd.DataFrame(Y_test, columns=df.columns[4:])
Y_pred = pd.DataFrame(Y_pred, columns=df.columns[4:])


for column in Y_test.columns:
    print(f'Classification Report for {column}:')
    print(classification_report(Y_test[column], Y_pred[column]))

Classification Report for related:
             precision    recall  f1-score   support

          0       0.60      0.35      0.44      1266
          1       0.81      0.93      0.86      3938
          2       0.64      0.17      0.27        40

avg / total       0.76      0.78      0.76      5244

Classification Report for request:
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.83      0.38      0.52       895

avg / total       0.88      0.88      0.86      5244

Classification Report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Classification Report for aid_related:
             precision    recall  f1-score   support

          0       0.73      0.88      0.79      3113
          1       0.74      0.52      0.61      2131


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
#version 1 - took 30 mins to run and hadn't completely run
parameters = {
    'clf__estimator__n_estimators': [50, 100],  
    'clf__estimator__min_samples_split': [2, 3],  
    'clf__estimator__max_depth': [None, 10]  
}

cv = GridSearchCV(model, param_grid=parameters,
                 cv=3,
                 n_jobs=-1,
                 verbose=2);
cv.fit(X_train, Y_train);

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__es

[Parallel(n_jobs=-1)]: Done   6 out of  24 | elapsed: 28.8min remaining: 86.5min


[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total=12.7min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total= 8.2min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total= 8.1min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, total=11.1min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, total=11.2min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__e

Process ForkPoolWorker-13:
Process ForkPoolWorker-14:


KeyboardInterrupt: 

In [12]:
parameters = {
    'clf__estimator__n_estimators': [10, 50],  
    'clf__estimator__min_samples_split': [2, 3],  
    'clf__estimator__max_depth': [None, 10]  
}

cv = GridSearchCV(model, param_grid=parameters,
                 cv=2,
                 n_jobs=-1,
                 verbose=2);
cv.fit(X_train, Y_train);

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimat

[Parallel(n_jobs=-1)]: Done   3 out of  16 | elapsed:  6.2min remaining: 26.8min


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 4.5min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total= 6.6min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total= 6.5min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 6.7min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 6.8min
[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, total= 8.2min
[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, total= 8.2min
[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 8.6min
[CV]  clf__estimator__max_depth=No

[Parallel(n_jobs=-1)]: Done  12 out of  16 | elapsed:  9.3min remaining:  3.1min


[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total=12.4min
[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total=12.5min
[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=13.0min
[CV]  clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=13.1min


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 13.5min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
Y_pred = cv.predict(X_test)

# Convert Y_test and Y_pred to DataFrames for easier column access
Y_test = pd.DataFrame(Y_test, columns=df.columns[4:])
Y_pred = pd.DataFrame(Y_pred, columns=df.columns[4:])


for column in Y_test.columns:
    print(f'Classification Report for {column}:')
    print(classification_report(Y_test[column], Y_pred[column]))

Classification Report for related:
             precision    recall  f1-score   support

          0       0.73      0.26      0.38      1266
          1       0.80      0.97      0.87      3938
          2       0.80      0.10      0.18        40

avg / total       0.78      0.79      0.75      5244

Classification Report for request:
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      4349
          1       0.88      0.45      0.59       895

avg / total       0.89      0.90      0.88      5244

Classification Report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Classification Report for aid_related:
             precision    recall  f1-score   support

          0       0.77      0.88      0.82      3113
          1       0.78      0.62      0.69      2131


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download(['punkt', 'wordnet'])

# Function to load data from SQLite database
def load_data(database_filepath):
    engine = create_engine(f'sqlite:///{database_filepath}')
    df = pd.read_sql_table('df', engine)

    X = df['message'].values  # Message column as input
    Y = df.iloc[:, 4:].values  # All categories as output
    category_names = df.columns[4:]  # Names of categories

    return X, Y, category_names

# Function to clean and tokenize text
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)

    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

    return clean_tokens

# Function to build a machine learning pipeline
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to count vectors
        ('tfidf', TfidfTransformer()),  # Apply TF-IDF transformation
        ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classification
    ])

    # Hyperparameter tuning using GridSearchCV
    parameters = {
        'clf__estimator__n_estimators': [50, 100],  # Number of trees in the forest
        'clf__estimator__min_samples_split': [2, 3]  # Minimum samples per split
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

    return cv

# Function to evaluate the model performance
def evaluate_model(model, X_test, Y_test, category_names):
    Y_pred = model.predict(X_test)

    print("\nClassification Report:\n")
    accuracy_list = []

    for i, category in enumerate(category_names):
        print(f"Category: {category}")
        print(classification_report(Y_test[:, i], Y_pred[:, i]))
        acc = accuracy_score(Y_test[:, i], Y_pred[:, i])
        accuracy_list.append(acc)
        print("-" * 60)

    overall_accuracy = np.mean(accuracy_list)
    print(f"\nOverall Model Accuracy: {overall_accuracy:.4f}")

# Function to save the trained model as a pickle file
def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as f:
        pickle.dump(model, f)

# Main execution function
def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]

        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

        print('Building model...')
        model = build_model()

        print('Training model...')
        model.fit(X_train, Y_train)

        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':python models/train_classifier.py data/DisasterResponse.db models/classifier.pkl
    main()